In [ ]:
# Mount Google Drive Folder (/content/drive/My Drive/Colab)
from google.colab import drive
drive.mount('/content/drive')

# import user defined files 
import sys
import_dir ="/content/drive/My Drive/Colab Notebooks/3. Semantic Segmentation"
sys.path.insert(0, import_dir)

# import ipynb files
!pip install import_ipynb 
import import_ipynb

# Import ipynb in different directory
# %run '/content/drive/My Drive/Colab Notebooks/3. Semantic Segmentation/dataset.ipynb'

# import easydict instead of using argparse (argparse is not usuable for ipynb)
import easydict 
global _args
_args = easydict.EasyDict({"config": '/content/drive/My Drive/Colab Notebooks/3. Semantic Segmentation/config/isic.yaml'})

# Debugging Tool
import pdb

In [ ]:
%cp -r '/content/drive/My Drive/Colab Notebooks/Dataset/ISIC/ISIC.zip' '/content/sample_data'
!unzip '/content/sample_data/ISIC.zip'

In [ ]:
train_root = "/content/ISIC_Trainset"
test_root = "/content/ISIC_Testset"

In [ ]:
import os
import pdb
import argparse
import torch

import models
from utils import dataset, config, utils, dataset_utils

In [ ]:
def get_parser(args):
    '''
    # Removed Code (Because ipynb file does not support argparse)
    # Even removed, these lines are substitued by easydict code

    parser = argparse.ArgumentParser(description='PyTorch Semantic Segmentation')
    parser.add_argument('--config', type=str, default='config/cityscapes.yaml')
    args = parser.parse_args()
    '''
    
    assert args.config is not None
    cfg = config.load_cfg_from_cfg_file(args.config)
    return cfg

In [ ]:
def check(args):
    assert args.data_name in ['crack', 'cityscapes', 'isic']
    assert args.classes >= 1
    assert args.model in ['unet32', 'unet64', 'unet128',
                          'deeplab34','deeplab50','deeplab101',
                          'pspnet18', 'pspnet34', 'pspnet50']
    assert args.loss in ['bce','ce','dice']

In [ ]:
def main():
    global args
    args = get_parser(_args)
    check(args)
    os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(str(x) for x in args.gpu)

    folders = [os.path.join(args.save_folder, 'prediction/gray'),
               os.path.join(args.save_folder, 'prediction/color')]
    os.makedirs(folders[0], exist_ok=True)
    os.makedirs(folders[1], exist_ok=True)

    _, transforms, _, postprocess, draw_result = dataset_utils.dataset_utils(args) #43:40
    test_loader = torch.utils.data.DataLoader(
                    dataset.Dataset(mode='test', data_root=args.test_root,
                                    data_list=args.test_list, transform=transforms[2]),
                    batch_size=args.batch_size, shuffle=False,
                    num_workers=args.workers, pin_memory=True)

    model = models.load_model(args)
    if len(args.gpu) > 1:
        model = torch.nn.DataParallel(model.cuda())
    else:
        model = model.cuda()

    if os.path.isfile(args.test_weight):
        ckpt = torch.load(args.test_weight,map_location=lambda storage,loc: storage.cuda())
        model.load_state_dict(ckpt['state_dict'], strict=True)
        print(f"=> loaded checkpoint {args.test_weight}")
    else:
        print(f"=> no weight found at {args.test_weight}")

    test(test_loader, model, postprocess, draw_result, folders)

In [ ]:
def test(test_loader, model, postprocess, draw_result, folders):
    print(f'\nMode: Test | Model: {args.model} | Data: {args.data_name}')

    torch.set_grad_enabled(False)
    model.eval()
    for i, data in enumerate(test_loader): #data에 이미지와 라벨 2개가 들어있는데..
        img = data[0].cuda() #가져올때 이미지만(data[0]) 가져온다.
        output = model(img)

        filenames = data[2]
        img = img.detach().cpu().numpy()
       
        pred = postprocess(output)        
        draw_result(img, pred, filenames, folders[1], overlap=True) 
        draw_result(img, pred, filenames, folders[0], overlap=False) 
        utils.progress_bar(i, len(test_loader), '')
    torch.cuda.empty_cache()

In [ ]:
if __name__ == '__main__':
    main()